In [1]:

# ═══════════════════════════════════════════════════════════════════════════
# CELL 1: Imports & Configuration
# ═══════════════════════════════════════════════════════════════════════════
import os, re, json, asyncio, random, shutil
from pathlib import Path
from datetime import datetime
from collections import defaultdict

import nest_asyncio
from dotenv import load_dotenv
from groq import AsyncGroq
from tqdm.notebook import tqdm

nest_asyncio.apply()

# Load .env from workspace root (2 levels up from this notebook)
_env_path = Path("../../.env")
if _env_path.exists():
    load_dotenv(_env_path)
    print(f"✅ .env loaded from {_env_path.resolve()}")
else:
    print(f"⚠️  .env not found at {_env_path.resolve()}, falling back to environment variables")

# ── Configuration ──────────────────────────────────────────────────────────
GROQ_API_KEY    = os.getenv("GROQ_API_KEY")

HELPCENTER_FILE = "cebu_pacific_helpcenter.txt"
TRAINSET_OUT    = "cebu_pacific_trainset_v2.jsonl"
VALSET_OUT      = "cebu_pacific_valset_v2.jsonl"

# Output dir for the optimization pipeline (copies will be placed here)
OPT_DIR = Path("../Customer_Ticket_Support_Agent")

# ⚡ Token-optimized: use fast 8B model (≈15× cheaper than 70B)
GENERATION_MODEL    = "llama-3.1-8b-instant"

# 📈 Enrichment settings — 5 diverse personas per article × 176 articles = 880 raw
QUERIES_PER_ARTICLE = 5        # 5 distinct persona types per article (was 2)
CONCURRENCY         = 3        # Conservative — stays within Groq rate limits
TRAIN_RATIO         = 0.80
MAX_BODY_CHARS      = 2000     # More article context → better-grounded resolutions (was 1200)

assert GROQ_API_KEY, "❌ GROQ_API_KEY not found — check your .env file"

est_raw = 176 * QUERIES_PER_ARTICLE
print("✅ Config ready")
print(f"   API Key  : {GROQ_API_KEY[:8]}...{GROQ_API_KEY[-4:]}")
print(f"   Model    : {GENERATION_MODEL}")
print(f"   Examples : {QUERIES_PER_ARTICLE} per article  (was 2)")
print(f"   Est. raw : ~{est_raw} examples  (was ~352)")
print(f"   Split    : {int(TRAIN_RATIO*100)}% train / {int((1-TRAIN_RATIO)*100)}% val")
print(f"   Max body : {MAX_BODY_CHARS} chars/article  (was 1200)")
print(f"   Opt dir  : {OPT_DIR.resolve()}")


✅ .env loaded from /Users/twotabs/Documents/GitHub/TwoTabs_James_Dev/.env
✅ Config ready
   API Key  : gsk_wtFg...yQtI
   Model    : llama-3.1-8b-instant
   Examples : 5 per article  (was 2)
   Est. raw : ~880 examples  (was ~352)
   Split    : 80% train / 19% val
   Max body : 2000 chars/article  (was 1200)
   Opt dir  : /Users/twotabs/Documents/GitHub/TwoTabs_James_Dev/DsPy-Learning-Repo/Customer_Ticket_Support_Agent


In [2]:

# ═══════════════════════════════════════════════════════════════════════════
# CELL 3: Parse helpcenter.txt → List of Article Dicts
# ═══════════════════════════════════════════════════════════════════════════
def parse_helpcenter(filepath: str) -> list[dict]:
    """
    Parse the master TXT file into individual article dicts.
    Handles the [001] TITLE / URL / CATEGORY / body structure.
    """
    text    = Path(filepath).read_text(encoding="utf-8")
    chunks  = re.split(r"={60,}", text)   # Split by ══ separator
    articles = []

    for chunk in chunks:
        chunk = chunk.strip()
        if not chunk or not re.match(r"\[\d+\]", chunk):
            continue

        lines = chunk.split("\n")

        # Header line: [001] Article Title
        header_match = re.match(r"\[(\d+)\]\s+(.+)", lines[0].strip())
        if not header_match:
            continue
        idx, title = header_match.groups()

        url = category = ""
        body_start_idx = 0

        for i, line in enumerate(lines[1:], 1):
            if line.startswith("URL"):
                url = re.split(r":\s+", line, maxsplit=1)[-1].strip()
            elif line.startswith("CATEGORY"):
                category = re.split(r":\s+", line, maxsplit=1)[-1].strip()
            elif re.match(r"^-{20,}$", line.strip()):
                body_start_idx = i + 1
                break

        body = "\n".join(lines[body_start_idx:]).strip()

        # Clean trailing noise (related articles section, feedback widget)
        body = re.sub(r"\n#{1,3}\s*Related articles.*$", "", body, flags=re.DOTALL)
        body = re.sub(r"Did you find this article helpful\?.*$", "", body, flags=re.DOTALL)
        body = re.sub(r"\n{3,}", "\n\n", body).strip()

        if len(body) < 50:    # Skip empty/stub articles
            continue

        articles.append({
            "idx":      int(idx),
            "title":    title.strip(),
            "url":      url,
            "category": category,
            "body":     body,
        })

    print(f"✅ Parsed {len(articles)} articles from {filepath}")

    by_cat = defaultdict(int)
    for a in articles:
        by_cat[a["category"]] += 1
    print(f"\n{'Category':<40} {'Articles':>8}")
    print("-" * 50)
    for cat, n in sorted(by_cat.items(), key=lambda x: -x[1]):
        print(f"  {cat:<38} {n:>8}")

    return articles


articles = parse_helpcenter(HELPCENTER_FILE)


✅ Parsed 176 articles from cebu_pacific_helpcenter.txt

Category                                 Articles
--------------------------------------------------
  Articles/Flying Out Soon                     38
  Booking Flights with Cebu Pacific            27
  Check-in & Boarding                          16
  Rebooking or Canceling Your Flight           15
  Booking a Flight                             14
  Prepare for Your Flight                       9
  Adding or Linking a Booking to Your Account        8
  Cancelations & Delays                         6
  Hi, how can we help?                          6
  Manage Your Booking                           6
  Add-ons & Services                            5
  Baggage & Cargo                               4
  Bali (Denpasar) Airport and Travel Guide        3
  Bangkok (Don Mueang) Airport and Travel Guide        3
  CEB Baggage Guidelines and Upgrade Options        3
  Signing up for a MyCebuPacific Account        3
  Suspension of Cebu Paci

In [3]:

# ═══════════════════════════════════════════════════════════════════════════
# CELL 4: Prompt Template & Async Generator  (diverse-persona, enriched)
# ═══════════════════════════════════════════════════════════════════════════
SYSTEM_PROMPT = (
    "You are a dataset engineer building a Cebu Pacific support AI training set. "
    "Output valid JSON only — no markdown fences, no commentary."
)

# Five distinct passenger personas to maximise query diversity
PERSONAS = """PERSONA GUIDE — use each persona exactly once across your {n} examples:
1. FRUSTRATED/URGENT  — ALL CAPS, multiple !!!, genuine panic (e.g. "flight is TOMORROW help!!!")
2. CONFUSED NEWCOMER  — first-time flyer, uncertain wording, asks very basic questions
3. TAGALOG-ENGLISH    — natural code-switch (e.g. "pwede ba mag-rebook", "magkano ang bayad")
4. POLITE FORMAL      — complete sentences, proper grammar, professional tone
5. POLICY CHALLENGE   — questions the fairness of a fee/rule, mild escalation language"""


def build_prompt(article: dict, n: int) -> str:
    body = article["body"][:MAX_BODY_CHARS]
    personas = PERSONAS.format(n=n)
    return f"""Generate {n} diverse customer support training examples from this Cebu Pacific help article.

CATEGORY: {article['category']}
TITLE: {article['title']}
CONTENT:
{body}

{personas}

Field rules:
- customer_query   : 50-200 chars. Must sound like a real passenger message for that persona.
                     Ask about something SPECIFIC in the article (not just restate the title).
- issue_category   : snake_case slug describing the problem (e.g. baggage_allowance_inquiry,
                     online_checkin_failure, refund_request, rebooking_fee_dispute)
- resolution       : 400-900 chars. Structured with bullet points or numbered steps.
                     Ground EVERY policy claim in the article content above.
                     Mention specific fees, timelines, or contact channels where present.
                     Close with a next-step or follow-up question for the passenger.
- resolution_time_minutes : integer 3-12

Return a JSON array of exactly {n} objects. Nothing else.
[{{"customer_query":"...","issue_category":"...","resolution":"...","resolution_time_minutes":5}}]"""


async def generate_for_article(
    client: AsyncGroq,
    article: dict,
    sem: asyncio.Semaphore,
    n: int = 5,
) -> list[dict]:
    """Generate n persona-diverse examples for one article with retry + backoff."""
    async with sem:
        for attempt in range(4):
            try:
                resp = await client.chat.completions.create(
                    model=GENERATION_MODEL,
                    messages=[
                        {"role": "system", "content": SYSTEM_PROMPT},
                        {"role": "user",   "content": build_prompt(article, n)},
                    ],
                    temperature=0.85,   # Higher temp → more varied phrasing across personas
                    max_tokens=2800,    # 5 examples × ~550 chars resolution = ~2750 needed
                )
                raw = resp.choices[0].message.content.strip()

                # Strip accidental markdown fences
                raw = re.sub(r"^```(?:json)?\s*", "", raw)
                raw = re.sub(r"\s*```$", "", raw)

                match = re.search(r"\[.*\]", raw, re.DOTALL)
                if not match:
                    raise ValueError("No JSON array found in response")

                examples = json.loads(match.group())

                for ex in examples:
                    ex["source_article"] = article["title"]
                    ex["source_url"]     = article["url"]
                    ex["category"]       = article["category"]

                return examples

            except Exception as e:
                if attempt == 3:
                    print(f"   ❌ Skipped [{article['idx']:03d}] "
                          f"{article['title'][:45]}: {str(e)[:60]}")
                wait = 2 ** attempt + random.uniform(0, 1.5)
                await asyncio.sleep(wait)

    return []


print("✅ Enriched prompt + generator ready")
print(f"   Model       : {GENERATION_MODEL}")
print(f"   Max tokens  : 2800 per call  (was 1400)")
print(f"   Max body    : {MAX_BODY_CHARS} chars  (was 1200)")
print(f"   Personas    : 5 distinct styles per article")
print(f"   Temperature : 0.85  (higher for style variety)")


✅ Enriched prompt + generator ready
   Model       : llama-3.1-8b-instant
   Max tokens  : 2800 per call  (was 1400)
   Max body    : 2000 chars  (was 1200)
   Personas    : 5 distinct styles per article
   Temperature : 0.85  (higher for style variety)


In [4]:
# ═══════════════════════════════════════════════════════════════════════════
# CELL 5: Run Async Batch Generation
# ═══════════════════════════════════════════════════════════════════════════
async def generate_full_dataset(articles: list[dict]) -> list[dict]:
    client     = AsyncGroq(api_key=GROQ_API_KEY)
    sem        = asyncio.Semaphore(CONCURRENCY)
    all_results = []

    estimated  = len(articles) * QUERIES_PER_ARTICLE
    print(f"🚀 Generating ~{estimated} examples from {len(articles)} articles")
    print(f"   Model: {GENERATION_MODEL}  |  Concurrency: {CONCURRENCY}\n")

    coros = [
        generate_for_article(client, art, sem, QUERIES_PER_ARTICLE)
        for art in articles
    ]

    for fut in tqdm(
        asyncio.as_completed(coros),
        total=len(coros),
        desc="Articles processed"
    ):
        results = await fut
        all_results.extend(results)

    print(f"\n✅ Raw generation complete: {len(all_results)} examples")
    return all_results


raw_examples = asyncio.run(generate_full_dataset(articles))


🚀 Generating ~880 examples from 176 articles
   Model: llama-3.1-8b-instant  |  Concurrency: 3



Articles processed:   0%|          | 0/176 [00:00<?, ?it/s]

   ❌ Skipped [008] Signing up for a MyCebuPacific Account: Invalid control character at: line 5 column 510 (char 684)
   ❌ Skipped [033] AirSWIFT Airlines Migrates to Cebu Pacific Sy: Expecting ',' delimiter: line 23 column 142 (char 2495)

✅ Raw generation complete: 873 examples


In [5]:

# ═══════════════════════════════════════════════════════════════════════════
# CELL 6: Quality Filter + Deduplication  (thresholds aligned to enriched prompt)
# ═══════════════════════════════════════════════════════════════════════════
def quality_filter(examples: list[dict]) -> list[dict]:
    """
    Keep examples that:
    - Have all required fields
    - Query is 40–250 chars (covers all 5 persona styles including long Tagalog-English)
    - Resolution is ≥300 chars (aligned with 400-900 char prompt target)
    - issue_category is a plausible snake_case slug (not blank, not a full sentence)
    - No near-duplicate queries (first 50 chars unique)
    """
    required = {"customer_query", "issue_category", "resolution", "resolution_time_minutes"}
    seen_queries = set()
    passed = []
    reasons = defaultdict(int)

    def is_valid_slug(s: str) -> bool:
        # Must look like snake_case, not a sentence
        return bool(re.match(r"^[a-z][a-z0-9_]{2,49}$", s))

    for ex in examples:
        if not required.issubset(ex.keys()):
            reasons["missing_fields"] += 1
            continue

        q = str(ex["customer_query"]).strip()
        r = str(ex["resolution"]).strip()
        cat = str(ex["issue_category"]).strip().lower().replace(" ", "_")

        # Length checks
        if not (40 <= len(q) <= 250):
            reasons["query_length"] += 1
            continue
        if len(r) < 300:
            reasons["resolution_too_short"] += 1
            continue

        # Category slug sanity
        if not is_valid_slug(cat):
            reasons["bad_issue_category"] += 1
            continue

        # Dedup on first 50 chars of query (wider window catches more near-dups)
        key = q[:50].lower()
        if key in seen_queries:
            reasons["duplicate"] += 1
            continue
        seen_queries.add(key)

        passed.append({
            "customer_query":          q,
            "issue_category":          cat,
            "resolution":              r,
            "resolution_time_minutes": int(ex.get("resolution_time_minutes", 5)),
            "source_article":          ex.get("source_article", ""),
            "source_url":              ex.get("source_url", ""),
            "category":                ex.get("category", ""),
        })

    print(f"✅ Quality filter: {len(examples)} → {len(passed)} examples kept")
    if reasons:
        print(f"   Removed reasons:")
        for reason, count in sorted(reasons.items(), key=lambda x: -x[1]):
            print(f"   - {reason:<25}: {count}")

    return passed


filtered_examples = quality_filter(raw_examples)


✅ Quality filter: 873 → 783 examples kept
   Removed reasons:
   - resolution_too_short     : 44
   - query_length             : 27
   - duplicate                : 19


In [6]:

# ═══════════════════════════════════════════════════════════════════════════
# CELL 7: Shuffle → Train/Val Split → Save as JSONL + Copy to Opt Pipeline
# ═══════════════════════════════════════════════════════════════════════════
def save_split(examples: list[dict], train_path: str, val_path: str) -> tuple:
    random.seed(42)        # Reproducible shuffle
    random.shuffle(examples)

    split_idx = int(len(examples) * TRAIN_RATIO)
    train_set = examples[:split_idx]
    val_set   = examples[split_idx:]

    def add_ids(dataset, prefix):
        for i, ex in enumerate(dataset, 1):
            ex["ticket_id"] = f"CP-2026-{prefix}-{i:04d}"
        return dataset

    train_set = add_ids(train_set, "TR")
    val_set   = add_ids(val_set,   "VL")

    def write_jsonl(path, data):
        Path(path).write_text(
            "\n".join(json.dumps(ex, ensure_ascii=False) for ex in data),
            encoding="utf-8"
        )

    # ── Save v2 files in this directory ────────────────────────────────────
    write_jsonl(train_path, train_set)
    write_jsonl(val_path,   val_set)

    # ── Copy to Customer_Ticket_Support_Agent so optimizer picks them up ───
    if OPT_DIR.exists():
        opt_train = OPT_DIR / "cebu_pacific_trainset.jsonl"
        opt_val   = OPT_DIR / "cebu_pacific_valset.jsonl"
        shutil.copy(train_path, opt_train)
        shutil.copy(val_path,   opt_val)
        print(f"   📋 Copied → {opt_train}")
        print(f"   📋 Copied → {opt_val}")
    else:
        print(f"   ⚠️  OPT_DIR not found ({OPT_DIR}), skipping copy")

    return train_set, val_set


trainset_v2, valset_v2 = save_split(filtered_examples, TRAINSET_OUT, VALSET_OUT)

print(f"\n✅ Dataset saved!")
print(f"   Train  : {len(trainset_v2):>4} examples → {TRAINSET_OUT}")
print(f"   Val    : {len(valset_v2):>4} examples → {VALSET_OUT}")
print(f"   Total  : {len(trainset_v2) + len(valset_v2):>4} examples")


   📋 Copied → ../Customer_Ticket_Support_Agent/cebu_pacific_trainset.jsonl
   📋 Copied → ../Customer_Ticket_Support_Agent/cebu_pacific_valset.jsonl

✅ Dataset saved!
   Train  :  626 examples → cebu_pacific_trainset_v2.jsonl
   Val    :  157 examples → cebu_pacific_valset_v2.jsonl
   Total  :  783 examples


In [7]:

# ═══════════════════════════════════════════════════════════════════════════
# CELL 8: Verification — Comprehensive Stats + Samples
# ═══════════════════════════════════════════════════════════════════════════
from collections import Counter

all_data = trainset_v2 + valset_v2

print("=" * 70)
print("📋 ENRICHED DATASET v2 SUMMARY")
print("=" * 70)
print(f"\n   Total examples : {len(all_data):,}")
print(f"   Train          : {len(trainset_v2):,}")
print(f"   Val            : {len(valset_v2):,}")

# ── Issue category distribution (top 25) ──────────────────────────────────
cat_counts = Counter(ex["issue_category"] for ex in all_data)
print(f"\n{'Issue Category':<40} {'Count':>6}  {'%':>5}")
print("-" * 55)
for cat, n in sorted(cat_counts.items(), key=lambda x: -x[1])[:25]:
    bar = "█" * int(n / len(all_data) * 200)
    print(f"  {cat:<38} {n:>6}  {n/len(all_data)*100:>4.1f}%  {bar}")

# ── Source article coverage ────────────────────────────────────────────────
sources = Counter(ex["source_article"] for ex in all_data)
print(f"\n   Unique source articles covered : {len(sources)}")
print(f"   Unique issue categories        : {len(cat_counts)}")

# ── Resolution quality ─────────────────────────────────────────────────────
lengths = [len(ex["resolution"]) for ex in all_data]
q_lengths = [len(ex["customer_query"]) for ex in all_data]
print(f"\n📊 Resolution Quality:")
print(f"   Avg length  : {sum(lengths) // len(lengths):,} chars")
print(f"   Min length  : {min(lengths):,} chars")
print(f"   Max length  : {max(lengths):,} chars")
print(f"\n📊 Query Length:")
print(f"   Avg length  : {sum(q_lengths) // len(q_lengths):,} chars")
print(f"   Min length  : {min(q_lengths):,} chars")
print(f"   Max length  : {max(q_lengths):,} chars")

# ── Sample from each set ───────────────────────────────────────────────────
for label, dataset in [("TRAIN", trainset_v2), ("VAL", valset_v2)]:
    sample = dataset[0]
    print(f"\n📝 {label} Sample:")
    print(f"   ticket_id      : {sample['ticket_id']}")
    print(f"   issue_category : {sample['issue_category']}")
    print(f"   source_article : {sample['source_article']}")
    print(f"   customer_query : {sample['customer_query']}")
    print(f"   resolution     : {sample['resolution'][:200].replace(chr(10), ' ')}...")

print(f"\n{'='*70}")
print(f"🎯 Files ready:")
print(f"   {TRAINSET_OUT}")
print(f"   {VALSET_OUT}")
if OPT_DIR.exists():
    print(f"   {OPT_DIR}/cebu_pacific_trainset.jsonl  ← optimization pipeline")
    print(f"   {OPT_DIR}/cebu_pacific_valset.jsonl    ← optimization pipeline")
print(f"{'='*70}")


📋 ENRICHED DATASET v2 SUMMARY

   Total examples : 783
   Train          : 626
   Val            : 157

Issue Category                            Count      %
-------------------------------------------------------
  rebooking_fee_dispute                     168  21.5%  ██████████████████████████████████████████
  baggage_allowance_inquiry                  64   8.2%  ████████████████
  online_checkin_failure                     35   4.5%  ████████
  refund_request                             34   4.3%  ████████
  rebooking_fee_inquiry                      25   3.2%  ██████
  visa_requirement_inquiry                    6   0.8%  █
  flight_status_inquiry                       5   0.6%  █
  baggage_fee_dispute                         4   0.5%  █
  rebooking_request                           4   0.5%  █
  online_checkin_fee_inquiry                  4   0.5%  █
  policy_challenge                            4   0.5%  █
  flight_change_request                       4   0.5%  █
  missing_itin